## 🛠️ **ETL (Extract, Transform, Load)**



####  **Importamos las librerías que vamos a usar**


In [1]:
import pandas as pd
import data_utils   
from data_utils import data_type_check, data_type_check_pkl

import warnings
warnings.filterwarnings('ignore')
import seaborn as sns
import matplotlib.pyplot as plt


#### 📦 **Extraccion** de los datos y primera exploración 


Mostrar info como en datatypecheck
#data_type_check_pkl('../0_Dataset/Yelp/business.pkl')

**PENDIENTE**
% de nulos, .shape, .columns


___

## Dataset Yelp

### business.pkl

In [2]:
business = pd.read_pickle('../0_Dataset/Yelp/business.pkl')
business.info()
business.sample(2)

<class 'pandas.core.frame.DataFrame'>
Index: 150346 entries, 0 to 150345
Data columns (total 28 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   business_id   150346 non-null  object
 1   name          150346 non-null  object
 2   address       150346 non-null  object
 3   city          150346 non-null  object
 4   state         150343 non-null  object
 5   postal_code   150346 non-null  object
 6   latitude      150346 non-null  object
 7   longitude     150346 non-null  object
 8   stars         150346 non-null  object
 9   review_count  150346 non-null  object
 10  is_open       150346 non-null  object
 11  attributes    136602 non-null  object
 12  categories    150243 non-null  object
 13  hours         127123 non-null  object
 14  business_id   5 non-null       object
 15  name          5 non-null       object
 16  address       5 non-null       object
 17  city          5 non-null       object
 18  state         5 non-null     

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,...,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
28610,QSn7eBGt3FCGiD9FwgSBuA,Sir Grout Nashville,"6339 Charlotte Pike, Ste 706",Nashville,PA,37209,36.144252,-86.872146,5.0,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
76454,xf3gVhy8TmNkzZgRbF5NOw,Red Apple Chinese Restaurant,6861 Jefferson Hwy,Harahan,PA,70123,29.937181,-90.19711,4.0,31,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Tenemos columnas duplicadas, procedemos a quitarlas.


In [3]:
business = business.loc[:, ~business.columns.duplicated()]
business.info()

<class 'pandas.core.frame.DataFrame'>
Index: 150346 entries, 0 to 150345
Data columns (total 14 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   business_id   150346 non-null  object
 1   name          150346 non-null  object
 2   address       150346 non-null  object
 3   city          150346 non-null  object
 4   state         150343 non-null  object
 5   postal_code   150346 non-null  object
 6   latitude      150346 non-null  object
 7   longitude     150346 non-null  object
 8   stars         150346 non-null  object
 9   review_count  150346 non-null  object
 10  is_open       150346 non-null  object
 11  attributes    136602 non-null  object
 12  categories    150243 non-null  object
 13  hours         127123 non-null  object
dtypes: object(14)
memory usage: 17.2+ MB


#### 🔁 **TRANSFORM**

Verificamos los nulos y duplicados de categories

In [4]:
# Cuenta los duplicados y nulos en la columna 'categories' antes de cualquier eliminación
duplicados_inicial = business['categories'].duplicated().sum()
nulos_inicial = business['categories'].isna().sum()

print("Antes: Cantidad de duplicados en 'categories':", duplicados_inicial)
print("Antes: Cantidad de NaNs en 'categories':", nulos_inicial)

# Elimina las filas con valores NaN en la columna 'categories'
business = business.dropna(subset=['categories'])

# Cuenta los duplicados y nulos en la columna 'categories' después de eliminar NaNs
duplicados_despues_nulos = business['categories'].duplicated().sum()
nulos_despues_nulos = business['categories'].isna().sum()

print("Después: Cantidad de NaNs en 'categories' ", nulos_despues_nulos)


'''
________________________________________________________________
FUTURO: Descomentar o borrar
Elimina las filas duplicadas basadas en la columna 'categories'
business = business.drop_duplicates(subset=['categories'])  
_________________________________________________________________  
'''

# Cuenta los duplicados en la columna 'categories' después de eliminar duplicados
duplicados_final = business['categories'].duplicated().sum()

print("Después: Cantidad de duplicados en 'categories'", duplicados_final)


Antes: Cantidad de duplicados en 'categories': 67185
Antes: Cantidad de NaNs en 'categories': 103
Después: Cantidad de NaNs en 'categories'  0
Después: Cantidad de duplicados en 'categories' 67083


Dado que diferentes negocios pueden compartir las mismas 'categories', no borramos los  67083 duplicados 


In [5]:
data_type_check(business)


 Resumen del dataframe:



Dimensiones:  (150243, 14)
         columna  %_no_nulos  %_nulos  total_nulos tipo_dato
0    business_id      100.00     0.00            0    object
1           name      100.00     0.00            0    object
2        address      100.00     0.00            0    object
3           city      100.00     0.00            0    object
4          state      100.00     0.00            3    object
5    postal_code      100.00     0.00            0    object
6       latitude      100.00     0.00            0    object
7      longitude      100.00     0.00            0    object
8          stars      100.00     0.00            0    object
9   review_count      100.00     0.00            0    object
10       is_open      100.00     0.00            0    object
11    attributes       90.92     9.08        13642    object
12    categories      100.00     0.00            0    object
13         hours       84.61    15.39        23120    object


#### **📤 LOAD**

In [6]:
#guardar en parquet
business.to_parquet("../0_Dataset/Yelp/business.parquet", engine="pyarrow")
